### 1. Installing "tweepy" library


In [ ]:
from IPython.display import clear_output
!pip install tweepy
clear_output(wait=False)

### 2. Importing required libraries

In [ ]:
import boto3
import os
import pandas as pd
import tweepy

### 3. Cofiguration
 - Bearer Token generated from the Twitter developer Project/App
 - S3 Bucket name

In [ ]:
token = "*"
bucket = "twitter-analytics-database"

### 4. Extracting tweets
1. Authenticating the client using bearer token
2. Communicating with @nytimes and extracting 100 recent tweets
    - Extracting tweet_id, time of tweet created and the tweet
3. Extracting the tweets from the resposne
4. Converting extracted data into a pandas dataframe
5. Saving the dataframe as a csv file in the S3 bucket

In [ ]:
# Helper function to save the data into a csv file
def save_to_csv(bucket, data):
    data = pd.DataFrame(data, columns=['id', 'created_at', 'tweet', 'tweet_length'])
    data.to_csv('tweets.csv', index=False, header=False)
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join('tweets.csv')).upload_file('tweets.csv')
    

# Helper function to extract tweets from @nytimes
def get_tweets(token, bucket):
    """
    To
        Authenticate with Bearer Token
        Extract the tweets
    
    Return
        100 recent tweets data
    """
    # Authenticating
    client = tweepy.Client(bearer_token=token)
    
    #  Communicating with @nytimes
    response = client.search_recent_tweets("nytimes", tweet_fields=["id", "created_at", "text"], max_results=100)
    
    # Extracting tweets from the response
    tweets = response.data
    
    data = [(tweet.id, tweet.created_at, tweet.text, len(tweet.text)) for tweet in tweets]
    
    save_to_csv(bucket, data)
    

get_tweets(token, bucket)